In [ ]:
SeqD = 512
InpD = OutD = SeqD
KernalS = 3
Bytes = 4
Order = 128
BatchS = 20

Memory = (InpD*OutD*KernalS + OutD)*Bytes*Order*BatchS
print('Approx ' + str(Memory/1000000000) + ' GB should used')

In [ ]:
# Assigning Values
device = torch.device("cuda")
input_dim = 512
d_model = 512
num_heads = 8
batch_size = 1000
sequence_length = 128
train_ratio = 0.8

# Initialising random input
x = torch.randn( (batch_size, sequence_length, input_dim), requires_grad = True ).to(device)

# Generate training data
torch.manual_seed(123)
model_mha = MultiheadAttention(input_dim, d_model, num_heads).to(device)
out = model_mha.forward(x)

x.size(): torch.Size([1000, 128, 512])
qkv.size(): torch.Size([1000, 128, 1536])
qkv.size(): torch.Size([1000, 128, 8, 192])
qkv.size(): torch.Size([1000, 8, 128, 192])
q size: torch.Size([1000, 8, 128, 64]), k size: torch.Size([1000, 8, 128, 64]), v size: torch.Size([1000, 8, 128, 64]), 
values.size(): torch.Size([1000, 8, 128, 64]), attention.size:torch.Size([1000, 8, 128, 128]) 
values.size(): torch.Size([1000, 128, 512])
out.size(): torch.Size([1000, 128, 512])


In [ ]:
for i in [2,4,8,16,32,64,128]:

    # Instantiating model
    model = HyenaOperator(
        d_model=d_model,
        l_max=sequence_length,
        order=i,
        filter_order=64
    ).to(device)

    # starting a new wandb run to track this script
    wandb.init(

        project="HyenaStats",

        config={
        "learning_rate": 0.001,
        "architecture": "Hyena",
        "dataset": "MHA Outputs",
        "epochs": 10,
        }
    )

    wandb.watch(model, log_freq=100)

    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Performing train-test split
    total_data = TensorDataset(x, out)
    train_size = int(0.8 * len(total_data))
    test_size = len(total_data) - train_size
    train_dataset, test_dataset = random_split(total_data, [train_size, test_size])

    # DataLoader for training and testing
    batch_size = 20
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward(retain_graph = True)
            optimizer.step()

            wandb.log({"loss": loss, "epoch": epoch})

        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, batch_idx+1, len(train_loader), loss.item()))

    # Testing the model
    model.eval()
    total_mse = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            mse = criterion(outputs, targets)
            total_mse += mse.item()

        average_mse = total_mse / len(test_loader)
        wandb.log({"error": average_mse})
        print('Mean Squared Error on the test set: {:.4f}'.format(average_mse))

    wandb.finish()

Epoch [1/10], Step [40/40], Loss: 0.0012
Epoch [2/10], Step [40/40], Loss: 0.0012
Epoch [3/10], Step [40/40], Loss: 0.0012
Epoch [4/10], Step [40/40], Loss: 0.0011
Epoch [5/10], Step [40/40], Loss: 0.0011
Epoch [6/10], Step [40/40], Loss: 0.0011
Epoch [7/10], Step [40/40], Loss: 0.0010
Epoch [8/10], Step [40/40], Loss: 0.0010
Epoch [9/10], Step [40/40], Loss: 0.0009
Epoch [10/10], Step [40/40], Loss: 0.0009
Mean Squared Error on the test set: 0.0013


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
error,▁
loss,█▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,9
error,0.00131
loss,0.00092


Epoch [1/10], Step [40/40], Loss: 0.0012
Epoch [2/10], Step [40/40], Loss: 0.0011
Epoch [3/10], Step [40/40], Loss: 0.0011
Epoch [4/10], Step [40/40], Loss: 0.0011
Epoch [5/10], Step [40/40], Loss: 0.0010
Epoch [6/10], Step [40/40], Loss: 0.0009
Epoch [7/10], Step [40/40], Loss: 0.0008
Epoch [8/10], Step [40/40], Loss: 0.0007
Epoch [9/10], Step [40/40], Loss: 0.0007
Epoch [10/10], Step [40/40], Loss: 0.0006
Mean Squared Error on the test set: 0.0015


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
error,▁
loss,█▅▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▃▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁
epoch,9
error,0.00146
loss,0.0006


Epoch [1/10], Step [40/40], Loss: 0.0012
Epoch [2/10], Step [40/40], Loss: 0.0012
Epoch [3/10], Step [40/40], Loss: 0.0011
Epoch [4/10], Step [40/40], Loss: 0.0011
Epoch [5/10], Step [40/40], Loss: 0.0010
Epoch [6/10], Step [40/40], Loss: 0.0011
Epoch [7/10], Step [40/40], Loss: 0.0012
Epoch [8/10], Step [40/40], Loss: 0.0016
Epoch [9/10], Step [40/40], Loss: 0.6165
Epoch [10/10], Step [40/40], Loss: 90654880.0000
Mean Squared Error on the test set: 1.6650


...

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
error,▁
loss,█▆▄▃▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,9
error,0.00119
loss,0.00118


Epoch [1/10], Step [40/40], Loss: 0.0014
Epoch [2/10], Step [40/40], Loss: 0.0012
Epoch [3/10], Step [40/40], Loss: 0.0012
Epoch [4/10], Step [40/40], Loss: 0.0012
Epoch [5/10], Step [40/40], Loss: 0.0012
Epoch [6/10], Step [40/40], Loss: 0.0012
Epoch [7/10], Step [40/40], Loss: 0.0011
Epoch [8/10], Step [40/40], Loss: 0.0012
Epoch [9/10], Step [40/40], Loss: 0.0012
Epoch [10/10], Step [40/40], Loss: 0.0012
Mean Squared Error on the test set: 0.0012


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
error,▁
loss,█▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,9
error,0.00119
loss,0.00118


Epoch [1/10], Step [40/40], Loss: 0.0014


OutOfMemoryError: ignored